In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
##########################################################################################
# UTILITIES CODE. RUN THIS CELL BEFORE EXECUTING ANYTHING ELSE IN THIS NOTEBOOK
##########################################################################################

# This ensures the same model, data and parâmeters produce repeatable results ############
import numpy as np
from numpy.random import seed
seed(1)
import tensorflow as tf
tf.random.set_seed(3)
##########################################################################################

import os
import matplotlib.pyplot as plt
import cv2 as cv


# IMAGE PROCESSING FUNCTIONS #############################################################

# Loads images from images_path folder and splits in train and test datasets 
def loadImages(images_path, batch_size, imgHeight, imgWidth):
    import tensorflow as tf

    train_set = tf.keras.preprocessing.image_dataset_from_directory(
        directory = images_path,
        labels="inferred",
        label_mode="categorical",
        class_names=None,
        batch_size = batch_size,
        color_mode = "rgb",
        image_size = (imgHeight, imgWidth),
        interpolation = "bilinear",
        seed = 700,
        shuffle = True,
        validation_split = 0.125,
        subset = 'training'
    )#.map(lambda x, y: (x/255.0, y) )

    test_set = tf.keras.preprocessing.image_dataset_from_directory(
        directory = images_path,
        labels="inferred",
        label_mode="categorical",
        class_names=None,
        batch_size = batch_size,
        color_mode = "rgb",
        image_size = (imgHeight, imgWidth),
        interpolation = "bilinear",
        seed = 700,
        shuffle = True,
        validation_split = 0.125,
        subset = 'validation'
    )#.map(lambda x, y: (x/255.0, y) )

    return( train_set, test_set)

def datasetToTensors(dataset):
    images = []
    labels = []
    for batch in dataset:
        (imagesBatch, labelsBatch) = batch
        images.append( imagesBatch )
        labels.append( labelsBatch )
    return( np.concatenate(images, axis=0 ), np.concatenate(labels, axis=0 ) )


# HEATMAP EXTRACTION FUNCTIONS ###########################################################
def predictHeatmaps(model, image, heatmapHeight, heatmapWidth):
    # Extracts an intermediate model that outputs heatmap logits
    heatmapsModel = getHeatmapsModel( 
        model, 
        1024, 1280 ) # These numbers are model summary Dense numbers 

    logitsList = heatmapsModel.predict(image)
    heatmaps = np.concatenate(logitsList)
    class0Heatmap = heatmaps[:,0].reshape(heatmapHeight,heatmapWidth)
    class1Heatmap   = heatmaps[:,1].reshape(heatmapHeight,heatmapWidth)
    return ( class0Heatmap, class1Heatmap )

def getHeatmapsModel(model, start, end):
    from keras import models
    return models.Model( inputs=model.input, outputs=getLogits(model,start, end) )

def getLogits(model, start, end):
    from keras import layers
    from keras import models
    return [ 
        model.get_layer('dense').output if i==0 
        else model.get_layer('dense_' + str(i)).output 
        for i in range(start,end)
    ]


# DISPLAY DATA FUNCTIONS #################################################################
def displayPredictionResults( model, 
                              images, labels, index, 
                              ratesColor, chartSize 
):
    print('IMAGE =>', index)
    print( 'Truth -> Dandelion' if labels[index][0] == 1 else 'Truth -> Sunflower' )  
    
    classProbabilitites, dandelionHmap, sunflowerHmap = getPredictions(
        model, images[index]  )

    displayPredictions( images[index], 
                        classProbabilitites, dandelionHmap, sunflowerHmap, 
                        ratesColor, chartSize   )
    print(' ')

def getPredictions(model, image):
    # model requires a batch of images as input, rescaled into range (0 - 1)
    imageBatch = np.reshape( image, (1, IM_HEIGHT, IM_WIDTH, 3) ) / 255.0

    # Predict class: Dandelion or Sunflower
    classProbabilitites = model.predict(imageBatch)
    
    # Gets heatmap predictions from model
    ( dandelionHmap, sunflowerHmap ) = predictHeatmaps(
            model, imageBatch, 
            int(IM_HEIGHT/PATCH_HEIGHT), int(IM_WIDTH / PATCH_WIDTH)  )

    return classProbabilitites, dandelionHmap, sunflowerHmap

def displayPredictions(
    image, 
    classProbabilitites, dandelionHmap, sunflowerHmap, 
    ratesColor, chartSize
):
    print(
        'Inference:  Dandelion=>', classProbabilitites[0,0], 
        '   Sunflower=>', classProbabilitites[0,1]  )
    
    # Rescales the raw heatmap into the 0 - 255 range
    rescaledHmap = rescaleHeatmap( dandelionHmap - sunflowerHmap )     
    
    # creates a (IM_HEIGHT, IM WIDTH, 3) image from the heatmap
    heatmapImage = heatmapToImage(rescaledHmap, IM_HEIGHT, IM_WIDTH, 
                                  PATCH_HEIGHT, PATCH_WIDTH)
    
    # Writes heatmap rates on the image
    imageBlended = blendImageAndRates( image, 
                                       heatmapToRange(rescaledHmap, -9, 9 ), 
                                       IM_HEIGHT, IM_WIDTH, PATCH_HEIGHT, 
                                       PATCH_WIDTH, ratesColor )
    # displays result image
    displayHeatmap( np.array( imageBlended, dtype=int ), heatmapImage, chartSize )

# Rescales the raw heatmap outputed by Dense layers into the 0 - 255 range, 
# with zero being mapped to 128
def rescaleHeatmap(heatmap):
    from sklearn.preprocessing import MinMaxScaler
    hmapMax = np.max(heatmap)  
    hmapMin = np.abs(np.min(heatmap))
    endOfScale = hmapMax if hmapMax > hmapMin else hmapMin
    scaler = 127.0 / endOfScale
    hmapScaled = np.array( (heatmap * scaler) + 127.0, dtype=int )
    hmapScaled[hmapScaled < 0] = 0
    return hmapScaled

# creates a (imgHeight, imgWidth, 3) rgb image from a heatmap (nRows, nCols)
def heatmapToImage(heatmap, imgHeight, imgWidth, patchHeight, patchWidth):
    image = np.zeros( (imgHeight, imgWidth), np.uint8)
    hmapHeight, hmapWidth = heatmap.shape
    nRows = int(imgHeight / patchHeight)
    nCols = int(imgWidth / patchWidth)
    for row in range(nRows):
        for col in range(nCols):
            image[(row*patchHeight):((row+1)*patchHeight), 
                  (col*patchWidth):((col+1)*patchWidth)] = int(heatmap[row,col])

    return image

# Converts heatmap values from the (0-255) range into a specified integer range
# start and end must be integers and (end > start) must hold True 
def heatmapToRange(heatmap, start, end):
    nRows, nCols = heatmap.shape
    rates = np.zeros((nRows,nCols))
    for row in range(nRows):
        for col in range(nCols):
            rates[row,col] = start + round((end-start) * heatmap[row,col] / 255.0)
    
    return rates

# Plots heatmap rates on the image
def blendImageAndRates( 
    imageRgb, rates, imgHeight, imgWidth, patchHeight, patchWidth, ratesColor
):
    # copy() needed because, otherwise, opencv would overwrite original tensor image
    image = imageRgb.copy()
    imgBgr = rgbToBgr(image)  # opencv assumes a bgr image, instead of rgb

    nRows = int(imgHeight / patchHeight)
    nCols = int(imgWidth / patchWidth)
    # Plots grid lines
    for row in range(1,nRows):
        cv.line( imgBgr, 
                ( 0,patchHeight*row ), ( imgWidth,patchHeight*row ), ratesColor, 1)
    for col in range(1,nCols):
        cv.line( imgBgr, 
                ( patchWidth*col,0 ), ( patchWidth*col,imgHeight ), ratesColor, 1)

    # Plots rates on image
    for row in range(0,nRows):
        for col in range(0,nCols):
            rate = int(rates[row,col])
            if( rate < 0 ):
                cv.putText( imgBgr, str(rate), 
                            ( patchWidth*col + 3, patchHeight*(row+1) - 6 ), 
                            cv.FONT_HERSHEY_SIMPLEX, 0.2, ratesColor, 1, cv.LINE_AA )
            else:
                cv.putText( imgBgr, str(rate), 
                            ( patchWidth*col + 7, patchHeight*(row+1) - 6 ), 
                            cv.FONT_HERSHEY_SIMPLEX, 0.2, ratesColor, 1, cv.LINE_AA )
    
    return bgrToRgb(imgBgr)

# Replace opencv functions here. These don't overwrite original images
# and accept dtypes float or int
def rgbToBgr(imgRgb):
    imgBgr = imgRgb.copy()
    imgBgr[:,:,0] = imgRgb[:,:,2] 
    imgBgr[:,:,1] = imgRgb[:,:,1] 
    imgBgr[:,:,2] = imgRgb[:,:,0]
    return imgBgr

def bgrToRgb(imgBgr):
    imgRgb = imgBgr.copy()
    imgRgb[:,:,0] = imgBgr[:,:,2] 
    imgRgb[:,:,1] = imgBgr[:,:,1] 
    imgRgb[:,:,2] = imgBgr[:,:,0]
    return imgRgb

# Displays the image and a heatmap
def displayHeatmap( image, heatmap, chart_size):
    import matplotlib.pyplot as plt
    f, ax = plt.subplots(figsize=(2*chart_size, chart_size))
    plt.subplot(1,2,1), plt.imshow(image, vmin=0, vmax=1.0)
    plt.title('ORIGINAL IMAGE WITH CLASS EVIDENCE'), plt.xticks([]), plt.yticks([])
    plt.xlabel('Positive numbers indicate Dandelion, negative indicate Sunflower')
    plt.subplot(1,2,2),plt.imshow(heatmap, cmap='viridis', vmin=0, vmax=255)
    plt.title('CLASS EVIDENCE HEATMAP'), plt.xticks([]), plt.yticks([])
    plt.xlabel('Yellow and Green indicate Dandelion, Blue and Purple indicate Sunflower')
    plt.show()



In [ ]:
IM_HEIGHT = 256
IM_WIDTH = 256
PATCH_HEIGHT = 16
PATCH_WIDTH = 16

BATCH_SIZE = 128

images_path = "/content/drive/My Drive/projects/ds_projects/explainable.dnn/images"

( train_set, test_set ) = loadImages(images_path, BATCH_SIZE, IM_HEIGHT, IM_WIDTH)
X_test, y_test = datasetToTensors(test_set)

print(X_test.shape)
print(y_test.shape)


Found 2048 files belonging to 2 classes.
Using 1792 files for training.
Found 2048 files belonging to 2 classes.
Using 256 files for validation.
(256, 256, 256, 3)
(256, 2)


In [ ]:
from keras.models import load_model

models_path = "/content/drive/My Drive/projects/ds_projects/explainable.dnn/models"
model_name = "/bagnets.model"

model = load_model(models_path + model_name, compile = True)
model.summary()

Streaming output truncated to the last 5000 lines.
 tf.__operators__.getitem_1152   (None, 16, 16, 3)   0           ['input_5[0][0]']                
 (SlicingOpLambda)                                                                                
                                                                                                  
 tf.__operators__.getitem_1153   (None, 16, 16, 3)   0           ['input_5[0][0]']                
 (SlicingOpLambda)                                                                                
                                                                                                  
 tf.__operators__.getitem_1154   (None, 16, 16, 3)   0           ['input_5[0][0]']                
 (SlicingOpLambda)                                                                                
                                                                                                  
 tf.__operators__.getitem_1155   (None, 16, 16, 3)   0    

In [ ]:
# Dandelion samples -> 3, 6, 10, 12, 13, 14, 255
ratesColor = (100,100,100)
displayPredictionResults( model, X_test, y_test, 0, ratesColor, 8 )   # Sunflower
displayPredictionResults( model, X_test, y_test, 9, (8,8,8), 8 )   # Sunflower
displayPredictionResults( model, X_test, y_test, 20, ratesColor, 8 )  # Sunflower

displayPredictionResults( model, X_test, y_test, 3, ratesColor, 8 )   # Dandelion
displayPredictionResults( model, X_test, y_test, 6, ratesColor, 8 )   # Dandelion
displayPredictionResults( model, X_test, y_test, 12, ratesColor, 8 )  # Dandelion
#displayPredictionResults( model, X_test, y_test, 255, ratesColor, 8 ) # Dandelion


Output hidden; open in https://colab.research.google.com to view.